<a href="https://colab.research.google.com/github/AVVSEE/AVVSEE/blob/main/concatenating_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook requires the original files to work correctly,
either add them to the colab environment or mount drive to use






# Dataframe **concatenation**

In [15]:
import pandas as pd

# List of filenames
files = ["abc_1.csv", "abc_2.csv", "abc_4.csv"]

# Initialize an empty list to store dataframes
dfs = []

# Iterate over files to read and append to the dfs list
for file in files:
    # Extract the file number
    number = int(file.split('_')[-1].split('.')[0])

    # Read the file with ISO-8859-1 encoding and ";" delimiter
    df = pd.read_csv(file, encoding='ISO-8859-1', delimiter=';')

    # Adding a new column with the file number
    df['store_id'] = number

    # Appending the dataframe to the dfs list
    dfs.append(df)

# Concatenating all dataframes into a single dataframe
final_df = pd.concat(dfs, ignore_index=True)

# Renaming columns for clarity
final_df = final_df.rename(columns={'Cod.Mercad': 'product_id'})

# Calculating new column **C_best**

In [16]:
# Function to determine the best label for each product based on the hierarchy A > B > C
def best_label(group):
    if 'A' in group.values:
        return 'A'
    elif 'B' in group.values:
        return 'B'
    else:
        return 'C'

# Applying the function to each group of product_id
C_best = final_df.groupby('product_id')['C'].apply(best_label)

# Merging the C_best series with the original dataframe based on the product_id
final_df = final_df.merge(C_best, on='product_id', how='left', suffixes=('', '_best'))

# **Dropping** unnecessary series

In [24]:
# Dropping unnecessary columns from the dataframe
final_df = final_df[['product_id', 'C', 'store_id', 'C_best']]

# Displaying the top rows of the updated dataframe
print(final_df.head(30))


    product_id  C  store_id C_best
0      12140.0  A         1      A
1      61284.0  A         1      A
2      61159.0  A         1      A
3      61177.0  A         1      A
4      33001.0  A         1      A
5      47568.0  A         1      A
6      91769.0  A         1      A
7      85954.0  A         1      A
8      26144.0  A         1      A
9      12783.0  A         1      A
10     96782.0  A         1      A
11     59395.0  A         1      A
12     89415.0  A         1      A
13     33029.0  A         1      A
14     61122.0  B         1      A
15     12774.0  B         1      A
16     32985.0  B         1      B
17     93455.0  B         1      A
18     96504.0  B         1      A
19     24645.0  B         1      A
20     54032.0  B         1      A
21     32994.0  B         1      B
22     98682.0  B         1      B
23     98664.0  B         1      B
24     98842.0  B         1      B
25     97433.0  B         1      A
26     78980.0  B         1      A
27     98174.0  B   

# **Checking** data integrity

In [19]:
final_df.shape

(20462, 4)

In [20]:
print(final_df.isnull().sum())
#no null values !

product_id    0
C             0
store_id      0
C_best        0
dtype: int64


In [21]:
print(final_df.dtypes)
#returns expected types

product_id    float64
C              object
store_id        int64
C_best         object
dtype: object


In [23]:
print(final_df['C'].unique())
#only expected values in calculated series


['A' 'B' 'C']
